In [3]:
%load_ext autoreload
%autoreload 2
from typing import *

In [4]:
from utils import load_input

# Part 1

The high-speed train leaves the forest and quickly carries you south. You can even see a desert in the distance! Since you have some spare time, you might as well see if there was anything interesting in the image the Mythical Information Bureau satellite captured.

After decoding the satellite messages, you discover that the data actually contains many small images created by the satellite's camera array. The camera array consists of many cameras; rather than produce a single square image, they produce many smaller square image tiles that need to be reassembled back into a single image.

Each camera in the camera array returns a single monochrome image tile with a random unique ID number. The tiles (your puzzle input) arrived in a random order.

Worse yet, the camera array appears to be malfunctioning: each image tile has been rotated and flipped to a random orientation. Your first task is to reassemble the original image by orienting the tiles so they fit together.

To show how the tiles should be reassembled, each tile's image data includes a border that should line up exactly with its adjacent tiles. All tiles have this border, and the border lines up exactly when the tiles are both oriented correctly. Tiles at the edge of the image also have this border, but the outermost edges won't line up with any other tiles.

By rotating, flipping, and rearranging them, you can find a square arrangement that causes all adjacent borders to line up:

To check that you've assembled the image correctly, multiply the IDs of the four corner tiles together. If you do this with the assembled tiles from the example above, you get 1951 * 3079 * 2971 * 1171 = 20899048083289.

Assemble the tiles into an image. What do you get if you multiply together the IDs of the four corner tiles?



In [5]:
raw_data = load_input(20, splitlines=False)

In [6]:
raw_tiles = raw_data.strip().split("\n\n")

In [77]:
from __future__ import annotations
from functools import cached_property

class Tile:
    def __init__(self, tile_id: int, grid: List[List[str]]):
        self.tile_id = tile_id
        self.grid = grid
        self.neighbors = dict()

    def rotate(self):
        self.grid = list(zip(*reversed(self.grid)))
    
    @cached_property
    def sides(self):
        out = set()
        for _ in range(4):
            out.add(tuple(self.grid[0]))
            out.add(tuple(reversed(self.grid[0]))) # must consider flipped
            self.rotate()
        return out
        
    @classmethod
    def from_raw(cls, s: str) -> Tile:
        header, tail = s.split("\n", 1)
        tile_id = int(header[5:9])
        grid = [list(row) for row in tail.splitlines()]
        return cls(tile_id, grid)
        

In [78]:
tiles = [Tile.from_raw(tile) for tile in raw_tiles]

In [79]:
tile_id_to_tile = {tile.tile_id:tile for tile in tiles}

In [80]:
len(tiles)

144

In [81]:
from itertools import combinations

In [82]:
for left, right in combinations(tiles, 2):
    common_sides = left.sides & right.sides
    for side in common_sides:
        left.neighbors[side] = right.tile_id
        right.neighbors[side] = left.tile_id

In [61]:
tiles[0].neighbors

{('.', '#', '#', '.', '.', '#', '.', '#', '.', '#'): 2239,
 ('.', '#', '.', '.', '.', '#', '.', '.', '#', '.'): 1621,
 ('#', '#', '.', '.', '.', '#', '.', '.', '.', '.'): 1997,
 ('#', '#', '#', '#', '.', '#', '.', '#', '#', '#'): 3701}

In [96]:
import math
math.prod([tile.tile_id for tile in tiles if len(tile.neighbors) == 4]) # pairs are duped under flips

140656720229539

## Part 2

Now, you're ready to check the image for sea monsters.

The borders of each tile are not part of the actual image; start by removing them.

In the example above, the tiles become:
```
.#.#..#. ##...#.# #..#####
###....# .#....#. .#......
##.##.## #.#.#..# #####...
###.#### #...#.## ###.#..#
##.#.... #.##.### #...#.##
...##### ###.#... .#####.#
....#..# ...##..# .#.###..
.####... #..#.... .#......

#..#.##. .#..###. #.##....
#.####.. #.####.# .#.###..
###.#.#. ..#.#### ##.#..##
#.####.. ..##..## ######.#
##..##.# ...#...# .#.#.#..
...#..#. .#.#.##. .###.###
.#.#.... #.##.#.. .###.##.
###.#... #..#.##. ######..

.#.#.### .##.##.# ..#.##..
.####.## #.#...## #.#..#.#
..#.#..# ..#.#.#. ####.###
#..####. ..#.#.#. ###.###.
#####..# ####...# ##....##
#.##..#. .#...#.. ####...#
.#.###.. ##..##.. ####.##.
...###.. .##...#. ..#..###
```
Remove the gaps to form the actual image:
```
.#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..######...
###.#####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#####.#
....#..#...##..#.#.###..
.####...#..#.....#......
#..#.##..#..###.#.##....
#.####..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..########.#
##..##.#...#...#.#.#.#..
...#..#..#.#.##..###.###
.#.#....#.##.#...###.##.
###.#...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.#..#.#
..#.#..#..#.#.#.####.###
#..####...#.#.#.###.###.
#####..#####...###....##
#.##..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###
```
Now, you're ready to search for sea monsters! Because your image is monochrome, a sea monster will look like this:
```
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
```
When looking for this pattern in the image, the spaces can be anything; only the # need to match. Also, you might need to rotate or flip your image before it's oriented correctly to find sea monsters. In the above image, after flipping and rotating it to the appropriate orientation, there are two sea monsters (marked with O):
```
.####...#####..#...###..
#####..#..#.#.####..#.#.
.#.#...#.###...#.##.O#..
#.O.##.OO#.#.OO.##.OOO##
..#O.#O#.O##O..O.#O##.##
...#.#..##.##...#..#..##
#.##.#..#.#..#..##.#.#..
.###.##.....#...###.#...
#.####.#.#....##.#..#.#.
##...#..#....#..#...####
..#.##...###..#.#####..#
....#.##.#.#####....#...
..##.##.###.....#.##..#.
#...#...###..####....##.
.#.##...#.##.#.#.###...#
#.###.#..####...##..#...
#.###...#.##...#.##O###.
.O##.#OO.###OO##..OOO##.
..O#.O..O..O.#O##O##.###
#.#..##.########..#..##.
#.#####..#.#...##..#....
#....##..#.#########..##
#...#.....#..##...###.##
#..###....##.#...##.##.#
```
Determine how rough the waters are in the sea monsters' habitat by counting the number of # that are not part of a sea monster. In the above example, the habitat's water roughness is 273.

How many # are not part of a sea monster?



In [97]:
[tile.tile_id for tile in tiles if len(tile.neighbors) == 6]

[1451,
 2927,
 2267,
 3989,
 1913,
 1009,
 1283,
 3779,
 1453,
 1129,
 2837,
 2617,
 3803,
 1823,
 1123,
 3659,
 3499,
 1733,
 2081,
 3517,
 3881,
 2539,
 2833,
 2243,
 2609,
 1321,
 2999,
 2251,
 1663,
 3709,
 2143,
 1223,
 1931,
 2203,
 3413,
 3169,
 2153,
 1789,
 3313,
 2719]

In [98]:
[tile.tile_id for tile in tiles if len(tile.neighbors) == 8]

[3469,
 3739,
 3581,
 2521,
 2729,
 2137,
 2179,
 2423,
 2017,
 1277,
 1999,
 2803,
 1801,
 3329,
 3719,
 2381,
 2593,
 3559,
 2953,
 2897,
 2693,
 1459,
 1109,
 3943,
 3299,
 1493,
 2879,
 2383,
 3373,
 3677,
 3457,
 1303,
 2687,
 1907,
 3217,
 3923,
 2239,
 2711,
 3019,
 3529,
 1367,
 2557,
 3391,
 2657,
 1429,
 2339,
 2347,
 2957,
 2477,
 1669,
 2887,
 3257,
 2029,
 1777,
 3919,
 1051,
 1297,
 1093,
 2843,
 1091,
 3527,
 1229,
 2549,
 2131,
 1201,
 1621,
 1483,
 1579,
 1511,
 3613,
 1997,
 2371,
 2621,
 1447,
 2551,
 2851,
 2129,
 1753,
 1471,
 1019,
 3467,
 2293,
 2647,
 2819,
 2749,
 2309,
 1697,
 1031,
 1097,
 3701,
 3617,
 3727,
 1847,
 3557,
 1867,
 2039,
 3301,
 1213,
 2437,
 3511]

In [100]:
tile_id_to_tile[3511].neighbors

{('.', '.', '#', '.', '.', '.', '#', '.', '.', '.'): 3559,
 ('.', '.', '.', '#', '.', '.', '.', '#', '.', '.'): 3559,
 ('.', '#', '#', '.', '.', '#', '#', '.', '#', '.'): 3299,
 ('.', '#', '.', '#', '#', '.', '.', '#', '#', '.'): 3299,
 ('.', '.', '.', '.', '.', '.', '.', '#', '.', '.'): 2999,
 ('.', '.', '#', '.', '.', '.', '.', '.', '.', '.'): 2999,
 ('.', '#', '#', '.', '#', '#', '#', '.', '#', '.'): 2819,
 ('.', '#', '.', '#', '#', '#', '.', '#', '#', '.'): 2819}